In [1]:
import pandas as pd
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.decomposition import RandomizedPCA

In [3]:
in_file = '/Public/BigPixel/Features Extracted/UCMerced/Finetune/Concat/UCM-ResNet51-fixed-finetune-1000-fc1001-fc1000.csv'
out_file = "../results/" + in_file.split("/")[-1].split(".csv")[0] + ".npy"
df = pd.read_csv(in_file)

In [4]:
df.head()

,0,0.1_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,...,990,991,992,993,994,995,996,997,998,999
0,agricultural00.tif,-5.985988,-2.632875,-1.239275,0.981469,2.181983,-0.937268,-0.293290,-2.421835,5.039817,...,-0.240605,-1.108775,-0.039925,0.465008,-1.036976,-0.523020,1.514892,-0.742436,2.691406,0.633042
1,agricultural01.tif,-4.734846,-1.385766,-0.808036,-1.598906,1.323177,-1.815063,-1.025150,-0.402246,3.692932,...,-2.821346,-1.345883,0.085303,-1.598293,-1.674048,-0.367275,0.478812,-1.457259,1.928455,3.155661
2,agricultural02.tif,-5.367169,-4.350185,1.270621,-3.720010,2.916455,2.639628,-1.354328,-0.552722,6.499020,...,-0.983849,-0.987947,-1.310067,-0.884034,-1.277995,0.271307,1.337530,-1.951130,1.679827,0.808132
3,agricultural03.tif,-5.413073,-1.857112,-0.298476,-1.561651,1.720369,0.005885,-0.228317,-0.125035,2.160454,...,-1.854178,-1.503122,0.253859,-0.822789,-1.408132,-0.765813,0.737210,-1.251438,2.036563,2.590920
4,agricultural04.tif,0.837424,-6.176835,3.070347,-2.667214,2.153282,4.907008,-0.521865,-2.203057,4.639434,...,1.579164,1.093745,-2.591939,-0.655340,1.109855,3.877506,4.635158,-3.434886,0.698258,1.599552


In [5]:
df = df.reindex(np.random.permutation(df.index))

df.head()

,0,0.1_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,...,990,991,992,993,994,995,996,997,998,999
1483,overpass83.tif,-5.092795,-4.323410,-8.093110,-0.570455,4.821430,9.224036,-8.468239,3.245510,5.633328,...,-5.903979,-6.114616,-4.523384,-8.152966,-6.904293,-5.336998,-2.674910,-6.737684,-0.729528,4.865066
1755,runway55.tif,-3.001887,-5.327291,-0.992837,-1.237405,2.687583,3.639000,-4.016954,2.007850,4.087449,...,-1.104352,-3.625035,-1.241803,-5.247230,-5.007086,-4.238493,-2.951863,-3.321991,-0.503330,4.294188
1173,intersection73.tif,0.714707,-6.534633,2.810425,-1.797145,2.245310,6.676893,-4.194223,4.245513,0.538878,...,2.309725,-4.771924,-2.488456,-5.012335,-3.484069,-1.991385,-2.970977,-5.131673,1.174945,2.279248
1447,overpass47.tif,-2.899192,-3.204572,-5.608658,-1.222683,2.855041,5.915286,-7.010575,-1.149062,3.568789,...,-3.891545,-5.305041,-4.195081,-5.997907,-4.172110,-2.113411,0.100377,-3.549310,2.594893,2.880086
1913,storagetanks13.tif,-3.957778,-8.926714,0.109093,0.942578,4.426417,5.954741,-6.834038,4.862585,5.034209,...,1.432020,-2.381471,-1.905261,-6.316472,-3.354561,-3.236649,-1.383633,-4.670828,0.507948,3.735312


In [6]:
X = np.array(df.iloc[:,1:])

import re
def getdigit(text):
    return re.compile('\d+').findall(text)
labels = [i.split(getdigit(i)[0])[0] for i in df.iloc[:,0]]

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = le.fit_transform(labels)

print "feature data is shape", X.shape
print "label data is shape", y.shape

feature data is shape (2100, 1344)
label data is shape (2100,)


In [7]:
pipe_svc = Pipeline([('sc1', StandardScaler()),
                    ('clf', SVC(kernel='linear'))])

param_range = np.logspace(-10, 1, 100)
param_grid = [{'clf__C': param_range}]
gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, cv=10, n_jobs=14)

In [8]:
skf = StratifiedKFold(y, n_folds=5)


In [ ]:
scores = []
for train_index, test_index in skf:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    gs.fit(X_train, y_train)
    clf = SVC(kernel= 'linear', C = gs.best_params_.values()[0])
    clf.fit(X_train, y_train) 
    scores.append(clf.score(X_test, y_test))

In [ ]:
scores

In [ ]:
np.mean(scores)

In [ ]:
gs.best_params_.values()[0]

In [ ]:
np.save(out_file, scores)

In [ ]:
import collections

In [ ]:
counter=collections.Counter(labels)
counter